<a href="https://colab.research.google.com/github/ShahdTarek/Football-Score-Detector/blob/main/Football_Score_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import cv2
from ultralytics import YOLO


def get_lower_upper(hsv_img , inp):

    if inp.split(',')[1]=='blue' :
        lower = np.array([100, 50, 50])
        upper = np.array([130, 255, 255])
        mask = cv2.inRange(hsv_img, lower_blue, upper_blue)

    elif inp.split(',')[1] =='red':
        lower_red = np.array([0, 50, 50])
        upper_red = np.array([10, 255, 255])
        red_mask1 = cv2.inRange(hsv_img, lower_red, upper_red)

        lower_red = np.array([170, 50, 50])
        upper_red = np.array([180, 255, 255])
        red_mask2 = cv2.inRange(hsv_img, lower_red, upper_red)

        # Combine the masks
        mask = red_mask1 + red_mask2

    elif inp.split(',')[1] =='green':
        lower_green = np.array([50, 50, 50])
        upper_green = np.array([70, 255, 255])
        mask = cv2.inRange(hsv_img, lower_green, upper_green)

    elif inp.split(',')[1] =='white':
        lower_white = np.array([0, 0, 180])
        upper_white = np.array([255, 25, 255])
        mask = cv2.inRange(hsv_img, lower_white, upper_white)

    else :
        print (' Please, Enter team Name and Color in format:e.g.(Ahly,red) ')
    return mask



ball = YOLO("/content/ball_best.pt")
person = YOLO("yolov8n.pt")
goal = YOLO("/content/goal_best.pt")


cap = cv2.VideoCapture("WhatsApp Video 2023-04-13 at 03.07.12.mp4")


inp = input(' Please, Enter FIRST team Name and Color in format:e.g.(Ahly,red) ')
inp2 = input(' Please, Enter SECOND team Name and Color in format:e.g.(Zamalek,white) ')

team_1_goal = inp.split(',')[0]
team_2_goal = inp2.split(',')[0]

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

# Below VideoWriter object will create
# a frame of above defined The output
# is stored in 'filename.mp4' file.
OutVid = cv2.VideoWriter('Slowed Frame Out.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, size)

j=0
i = 0
while True:
    ret, frame = cap.read()

    if ret:

        ball_results = ball.predict(frame)
        goal_results = goal.predict(frame)

        ##################################################################
        goal_results = goal_results[0]

        gboxes = goal_results.boxes

        data1 = pd.DataFrame(gboxes.boxes.numpy(), columns = ["Goal_x1", "Goal_y1", "Goal_x2", "Goal_y2", "conf1", "class1"])


        #################################################################

        result2 = ball_results[0]

        boxes2 = result2.boxes

        data2 = pd.DataFrame(boxes2.boxes.numpy(), columns =  ["Ball_x1","Ball_y1","Ball_x2","Ball_y2","conf2","class2"])

        BallCX = (( data2["Ball_x1"] + data2["Ball_x2"]) / 2)
        data1["Ball_center_x"] =BallCX


        BallCY = (( data2["Ball_y1"] + data2["Ball_y2"]) / 2)
        data1["Ball_center_y"] =BallCY


        ##################################################################

        Goal_start_point = (int(data1['Goal_x1'][0]),int(data1['Goal_y1'][0]))
        Goal_end_point = (int(data1['Goal_x2'][0]),int(data1['Goal_y2'][0]))

        res_plotted = ball_results[0].plot()

        res_plotted = cv2.rectangle(res_plotted, Goal_start_point, Goal_end_point, (255,0,0), 4)
        res_plotted = cv2.putText(res_plotted, 'Goal',( int(data1['Goal_x1'][0]) , int(data1['Goal_y1'][0])-10 ), cv2.FONT_HERSHEY_SIMPLEX , 1.3 , (255,0,0) , 2)



        if i == 0 :
            player_results = person.predict(frame, conf =0.4)

            boxes1 = player_results[0].boxes

            data = pd.DataFrame(boxes1.boxes.numpy(), columns = ["player_x1", "player_y1", "player_x2", "player_y2", "conf", "class"])

            data["player_center_x"] =( data["player_x1"] + data["player_x2"]) / 2
            data["player_center_y"] =( data["player_y1"] + data["player_y2"]) / 2
            data["Ball_center_x"] =BallCX
            data["Ball_center_y"] =BallCY

            data['distance'] = np.linalg.norm(data.iloc[:, [6,7]].values - data.loc[:, ['Ball_center_x','Ball_center_y']].values[0], axis=1)

            # Sort the dataframe by distance
            df_sorted = data.sort_values(by='distance')
            df_sorted["Ball_center_x"].fillna(df_sorted["Ball_center_x"][0], inplace = True)
            df_sorted["Ball_center_y"].fillna(df_sorted["Ball_center_y"][0], inplace = True)

            # Select the first row of the sorted dataframe
            closest_player = df_sorted.iloc[0]

            closest_player_x1 = int(df_sorted.head(1)['player_x1'].values)
            closest_player_y1 = int(df_sorted.head(1)['player_y1'].values)
            closest_player_x2 = int(df_sorted.head(1)['player_x2'].values)
            closest_player_y2 = int(df_sorted.head(1)['player_y2'].values)


            crop = frame [ closest_player_y1 : closest_player_y2 , closest_player_x1 : closest_player_x2 ]
            print(crop.shape)

            crop = cv2.cvtColor(crop,cv2.COLOR_BGR2HSV)
            mask1 = get_lower_upper(crop , inp)
            mask2 = get_lower_upper(crop , inp2)

            mask1_result = cv2.bitwise_and(crop, crop, mask = mask1)
            mask2_result = cv2.bitwise_and(crop, crop, mask = mask2)

            ratio1 = cv2.countNonZero(mask1)/(crop.size/3)
            ratio2 = cv2.countNonZero(mask2)/(crop.size/3)

            # This is the color percent calculation, considering the resize I did earlier.
            Team1_Percent = (ratio1 * 100) / 0.3
            Team2_Percent = (ratio2 * 100) / 0.3

            # Print the color percent, use 2 figures past the decimal point
            print('Team 1 percentage:', np.round(Team1_Percent, 2))
            print('Team 2 percentage:', np.round(Team2_Percent, 2))

            df_sorted['Team 1 percentage'] = np.round(Team1_Percent, 2)
            df_sorted['Team 2 percentage'] = np.round(Team2_Percent, 2)

            nearestP_start_point = (int(closest_player['player_x1']),int(closest_player['player_y1']))
            nearestP_end_point = (int(closest_player['player_x2']),int(closest_player['player_y2']))

            if df_sorted['Team 1 percentage'] .values[0] > df_sorted['Team 2 percentage'] .values[0]:
                res_plotted = cv2.rectangle(res_plotted, nearestP_start_point, nearestP_end_point, (100,255,255), 4)
                cv2.putText(res_plotted, f'{team_1_goal} Player',(int(closest_player['player_x1']) , int(closest_player['player_y1'])-10 ), cv2.FONT_HERSHEY_SIMPLEX , 1.3 , (100,255,255) , 2)
            else:
                cv2.putText(res_plotted, f'{team_2_goal} Player',(int(closest_player['player_x1']) , int(closest_player['player_y1'])-10 ), cv2.FONT_HERSHEY_SIMPLEX , 1.3 , (100,255,255) , 2)

            i+=1


        if data1['Ball_center_x'].isnull().values.any() == False:
            if int(data1['Ball_center_x']) in range(int(data1['Goal_x1']),int(data1['Goal_x2'])):
                if int(data1['Ball_center_y']) in range(int(data1['Goal_y1']),int(data1['Goal_y2'])):
                    if df_sorted['Team 1 percentage'] .values[0] > df_sorted['Team 2 percentage'] .values[0]:
                        print(f'=====================Goooooal For {team_1_goal}!!=====================')
                        cv2.putText(res_plotted, f'Goooooooooall For {team_1_goal}!',(50,680), cv2.FONT_HERSHEY_SIMPLEX , 1.5 , (0, 0, 255) , 3)
                    else:
                        print(f'=====================Goooooal For {team_2_goal}!!=====================')
                        cv2.putText(res_plotted, f'Goooooooooall For {team_2_goal}!',(50,680), cv2.FONT_HERSHEY_SIMPLEX , 1.5 , (0, 0, 255) , 3)
            else :
                print ('Out')
                cv2.putText(res_plotted, 'OUT',(50,680), cv2.FONT_HERSHEY_SIMPLEX , 1.5 , (0, 0, 255) , 3)


        while j < 50:
            OutVid.write(res_plotted)
            j+=1

        if i < 3:
            OutVid.write(res_plotted)
            cv2.waitKey(3000)
            i+=1
        else:
            OutVid.write(res_plotted)
            cv2.waitKey(1000)

        OutVid.write(res_plotted)
        cv2.imshow ('frame' , res_plotted )

        if cv2.waitKey(1000) & 0xff == ord('q'):
            break


cap.release()
OutVid.release()
cv2.destroyAllWindows()